In [ ]:
!pip install requests onnx onnxruntime transformers datasets accelerate sentencepiece matplotlib


In [ ]:
!pip install torchvision

In [ ]:
import requests
import torch
import torch.nn as nn

import onnx
import onnxruntime
import onnxruntime as ort
import numpy as np
import json
import io
import sys
import base64
from torch.utils.data import Dataset
from typing import Tuple
import pickle
import os
import torchvision.transforms as transforms

import pickle
import random
from torchvision import transforms
from PIL import Image

In [4]:
### REQUESTING NEW API ### 
TOKEN = "61729223" 
 
response = requests.get("http://34.122.51.94:9090" + "/stealing_launch", 
headers={"token": TOKEN}) 
answer = response.json() 
 
print(answer) 
if 'detail' in answer: 
    sys.exit(1) 
 
# save the values 
SEED = str(answer['seed']) 
PORT = str(answer['port']) 

{'seed': 36604352, 'port': '9340'}


{'seed': 36604352, 'port': '9340'}

In [5]:
class TaskDataset(Dataset):
    def __init__(self, transform=None):

        self.ids = []
        self.imgs = []
        self.labels = []

        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if not self.transform is None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)


In [6]:
mean = [0.2980, 0.2962, 0.2987]
std = [0.2886, 0.2875, 0.2889]

transform = transforms.Compose([
    transforms.Normalize(mean=mean, std=std),
])
dataset: TaskDataset = torch.load("ModelStealingPub.pt", weights_only=False)
dataset.transform = transform

In [ ]:
import io
import json
import base64
import requests
from typing import List, Dict

def model_stealing(ids: List[int], imgs: List, port: int) -> List[Dict]:
    endpoint = "/query"
    url = f"http://34.122.51.94:{port}{endpoint}"
    
    image_data = []
    for img in imgs:
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        img_base64 = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
        image_data.append(img_base64)

    payload = json.dumps(image_data)
    response = requests.get(url, files={"file": payload}, headers={"token": TOKEN})

    if response.status_code == 200:
        representations = response.json()["representations"]
        output = [
            {"id": id_, "img": img, "label": rep}
            for id_, img, rep in zip(ids, imgs, representations)
        ]
        return output
    else:
        raise Exception(
            f"Model stealing failed. Code: {response.status_code}, content: {response.json()}"
        )


Querying data from the vitcim model in batch of 1000s with time delay

In [31]:
import time
import numpy as np

batch_size = 1000
all_records = []

# Shuffle and batch
indices = np.random.permutation(len(dataset))

for start in range(0, len(indices), batch_size):
    end = start + batch_size
    batch_indices = indices[start:end]

    batch_ids = [dataset.ids[i] for i in batch_indices]
    batch_imgs = [dataset.imgs[i] for i in batch_indices]

    try:
        batch_output = model_stealing(batch_ids, batch_imgs, port=PORT)
        all_records.extend(batch_output)
        print(f"Batch {start // batch_size + 1} processed successfully.")
    except Exception as e:
        print(f"Error on batch {start // batch_size + 1}: {e}")
    
    # Wait 90 seconds (1.5 minutes) between requests
    if end < len(indices):  # Avoid waiting after last batch
        print("Waiting 90 seconds to respect API rate limit...")
        time.sleep(90)


Batch 1 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 2 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 3 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 4 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 5 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 6 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 7 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 8 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 9 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 10 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 11 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 12 processed successfully.
Waiting 90 seconds to respect API rate limit...
Batch 13 processed successfully.


In [ ]:
print(f"\nTotal records collected: {len(all_records)}\n")

# Print few samples
for i, record in enumerate(all_records[:5]):  
    print(f"Record {i+1}:")
    print(f"  ID    : {record['id']}")
    print(f"  Image : {record['img']}")  
    print(f"  Label : {record['label'][:5]}...")  
    print()


In [34]:
import pickle

with open("outv1.pickle", "wb") as f:
    pickle.dump(all_records, f)

print("Data saved to out.pickle")


Data saved to out.pickle


In [36]:
with open("outv1.pickle", "rb") as f:
    loaded_records = pickle.load(f)

print(f"Loaded {len(loaded_records)} records from out.pickle")


Loaded 13000 records from out.pickle


In [ ]:



# Define augmentation pipeline (no resize)
augmentation = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=3)
    ], p=0.3),
])

# Augment records
augmented_records = []

for record in loaded_records:
    for _ in range(4):
        img = record["img"]
        if not isinstance(img, Image.Image):
            img = Image.fromarray(img)

        # Apply augmentation
        aug_img = augmentation(img)

        # Create new record
        new_record = {
            "id": record["id"],
            "img": aug_img,
            "label": record["label"]
        }
        augmented_records.append(new_record)

print(f"Original records: {len(loaded_records)}")
print(f"Augmented records: {len(augmented_records)} (4x per original)")


Original records: 13000
Augmented records: 52000 (4x per original)


In [38]:
with open("augmented_outv1.pickle", "wb") as f:
    pickle.dump(augmented_records, f)

print("Augmented data saved to augmented_out.pickle")


Augmented data saved to augmented_out.pickle


In [2]:
# Load the augmented data
with open("augmented_outv1.pickle", "rb") as f:
    augmented_records = pickle.load(f)

print(f"Loaded {len(augmented_records)} augmented records.")


Loaded 52000 augmented records.


training

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


True
12.1


In [4]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
import random
import pickle
from collections import defaultdict
from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from tqdm import tqdm

# ========= 1. Load augmented data =========
with open("/content/augmented_outv1.pickle", "rb") as f:
    augmented_records = pickle.load(f)

# ========= 2. Keep only TWO samples per unique ID =========
records_by_id = defaultdict(list)
for rec in augmented_records:
    records_by_id[rec["id"]].append(rec)      

filtered_records = []
for lst in records_by_id.values():
    random.shuffle(lst)
    filtered_records.extend(lst[:2])          

# 80/20 random split
# ------------------------------------------------------------------------------
dataset_size = len(filtered_records)
train_len = int(0.8 * dataset_size)
test_len  = dataset_size - train_len
# ------------------------------------------------------------------------------
# Create Dataset class first and then:
# full_dataset = StolenDataset(filtered_records, transform)
# train_dataset, test_dataset = random_split(full_dataset, [train_len, test_len])


train_records, test_records = [], []
for lst in records_by_id.values():
    random.shuffle(lst)                       # shuffle again to avoid bias
    train_records.append(lst[0])              # one for training
    test_records.append(lst[1])               # one for testing
# ------------------------------------------------------------------------------

# ========= 3. Dataset class =========
class StolenDataset(Dataset):
    def __init__(self, records, transform=None):
        self.records = records
        self.transform = transform

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        rec = self.records[idx]
        img = rec["img"]                      
        if img.mode != "RGB":
            img = img.convert("RGB")
        label = torch.tensor(rec["label"], dtype=torch.float32)
        return self.transform(img) if self.transform else img, label

# ========= 4. Transform & DataLoaders =========
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.2980, 0.2962, 0.2987],
                         std=[0.2886, 0.2875, 0.2889]),
])

batch_size = 64
num_workers = 2

train_loader = DataLoader(
    StolenDataset(train_records, transform),  
    batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_loader = DataLoader(
    StolenDataset(test_records, transform),   
    batch_size=batch_size, shuffle=False, num_workers=num_workers
)

# ========= 5. Model, device, loss, optimiser =========
model = models.resnet18(weights=None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, 1024)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ========= 6. Training loop =========
epochs = 25
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = loss_fn(model(imgs), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1}: avg train loss = {running_loss / len(train_loader):.4f}")

# ========= 7. Quick test pass =========
model.eval()
test_loss = 0.0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        test_loss += loss_fn(model(imgs), labels).item()
print(f"\nTest loss (MSE): {test_loss / len(test_loader):.4f}")

# ========= 8. Save model =========
torch.save(model.state_dict(), "stolen_model.pth")
print("Model saved as stolen_model.pth")


Saved model as stolen_model.pth had to change it to onnx

In [ ]:
# -----------------------------------------------------------
# 1.  Set up Python imports
# -----------------------------------------------------------
import torch
from torch import nn
from torchvision import models
import onnx
import onnxruntime as ort
import numpy as np
from pathlib import Path

# -----------------------------------------------------------
# 2.  Rebuild the model architecture exactly as before
# -----------------------------------------------------------
model = models.resnet18(weights=None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, 1024)

# -----------------------------------------------------------
# 3.  Load your trained weights
# -----------------------------------------------------------
weights_path = "stolen_model.pth"            
state_dict   = torch.load(weights_path, map_location="cpu")
model.load_state_dict(state_dict)
model.eval()                                          

# -----------------------------------------------------------
# 4.  Export to ONNX with clear tensor names
# -----------------------------------------------------------
onnx_path = "stolen_model.onnx"        
dummy     = torch.randn(1, 3, 32, 32)                 

torch.onnx.export(
    model,
    dummy,
    onnx_path,
    opset_version=12,            
    input_names=["x"],           
    output_names=["features"],   
    dynamic_axes={
        "x":        {0: "batch"},    
        "features": {0: "batch"},
    },
)

print(f"ONNX model saved to {onnx_path}")

# -----------------------------------------------------------
# 5.  Sanity-check with onnxruntime 
# -----------------------------------------------------------
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
print("Input  name:", sess.get_inputs()[0].name)      
print("Output name:", sess.get_outputs()[0].name)     

test_out = sess.run(
    None, {"x": np.random.randn(1, 3, 32, 32).astype(np.float32)}
)[0][0]

assert test_out.shape == (1024,), f"Unexpected output shape: {test_out.shape}"
print("Dry-run successful — output shape:", test_out.shape)


In [ ]:
import numpy as np
import onnxruntime as ort

path = "stolen_model.onnx"
sess = ort.InferenceSession(path)

# Discover what the model actually expects
input_name  = sess.get_inputs()[0].name      
output_name = sess.get_outputs()[0].name     
print("Input  :", input_name)
print("Output :", output_name)

# Create a dummy batch and run
dummy = np.random.randn(1, 3, 32, 32).astype(np.float32)
out   = sess.run(None, {input_name: dummy})[0][0]  # drop batch dim
assert out.shape == (1024,), "Unexpected output shape"


Input  : x
Output : features


In [ ]:
SEED = str(36604352) 
PORT = str(9340) 
TOKEN = "61729223" 

with open(path, "rb") as f:
    model = f.read()
    try:
        stolen_model = ort.InferenceSession(model)
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")
    try:
        out = stolen_model.run(
            None, {"x": np.random.randn(1, 3, 32, 32).astype(np.float32)}
        )[0][0]
    except Exception as e:
        raise Exception(f"Some issue with the input, {e=}")
    assert out.shape == (1024,), "Invalid output shape"

# Send the model to the server
response = requests.post("http://34.122.51.94:9090/stealing", files={"file": open(path, "rb")}, headers={"token": TOKEN, "seed": SEED})
print(response.json())
